# Starlink SATCOM ISP Performance Monitoring with AI
## Author: Fatih E. NAR

In [ ]:
#%pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
from accelerate import Accelerator
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class StarlinkTransformer(nn.Module):
    def __init__(self, input_dim, num_heads=4, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=input_dim,
                nhead=num_heads,
                dim_feedforward=dim_feedforward,
                dropout=dropout
            ),
            num_layers=2
        )
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.transformer(x)
        x = torch.mean(x, dim=1)
        return self.fc(x)

In [ ]:
# Data preparation
df = pd.read_csv('data/starlink_data.csv')

# Display basic information about the dataset
df.info()

# Display the first few rows of the dataset
df.head()

label_encoders = {}
for cat_feat in ['season', 'weather']:
    label_encoders[cat_feat] = LabelEncoder()
    df[f'{cat_feat}_encoded'] = label_encoders[cat_feat].fit_transform(df[cat_feat])

features = ['latitude', 'longitude', 'elevation_m', 'visible_satellites', 
           'serving_satellites', 'signal_loss_db', 'season_encoded', 'weather_encoded']

X = df[features].values
y = df['qoe_score'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to tensors and create dataloaders
X_tensor = torch.FloatTensor(X_scaled)
y_tensor = torch.FloatTensor(y).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [ ]:
# Initialize accelerator
accelerator = Accelerator()

# Model setup
model = StarlinkTransformer(input_dim=X.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Variables
num_epochs = 200
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Prepare for accelerated training
model, optimizer, train_loader, test_loader = accelerator.prepare(
    model, optimizer, train_loader, test_loader
)

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X.unsqueeze(1))
        loss = criterion(outputs, batch_y)
        accelerator.backward(loss)
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for test_X, test_y in test_loader:
                y_pred = model(test_X.unsqueeze(1))
                test_loss += criterion(y_pred, test_y).item()
        print(f'Epoch {epoch+1}, Test Loss: {test_loss/len(test_loader):.4f}')

In [ ]:
# Save model and preprocessing
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
torch.save(unwrapped_model.state_dict(), 'models/starlink_transformer.pth')
joblib.dump(scaler, 'models/scaler.joblib')
joblib.dump(label_encoders, 'models/label_encoders.joblib')

In [ ]:
# Final evaluation
model.eval()
total_loss = 0
y_true = []
y_pred_list = []

with torch.no_grad():
    for test_X, test_y in test_loader:
        y_pred = model(test_X.unsqueeze(1))
        loss = criterion(y_pred, test_y)
        total_loss += loss.item()
        y_true.extend(test_y.cpu().numpy())
        y_pred_list.extend(y_pred.cpu().numpy())

final_loss = total_loss / len(test_loader)
y_true = np.array(y_true)
y_pred = np.array(y_pred_list)
r2 = 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)
print(f'\nFinal R² Score: {r2:.3f}')